In [22]:
import os
import pandas as pd
from weasyprint import HTML, CSS
import glob

# --- Configuration ---
# Use a raw string for Windows paths to avoid issues with backslashes
INPUT_DIR = r"D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database" # Renamed to reflect it handles more types
OUTPUT_PDF_DIR = "output_pdf_files"  # Folder where PDFs will be saved
RECURSIVE_SEARCH = True
CREATE_SUBFOLDERS_IN_OUTPUT = True
FILE_EXTENSIONS_TO_CONVERT = [".xlsx", ".csv"] # Specify which file types to look for

# --- CHOOSE PDF OUTPUT STRATEGY FOR XLSX (Does not affect CSVs much) ---
# OPTION_COMBINE_SHEETS_INTO_ONE_PDF:
#   True: For XLSX files with multiple sheets, combine all sheets into a single PDF.
#   False: For XLSX files with multiple sheets, create a separate PDF for each sheet.
# CSV files will always produce a single PDF as they don't have sheets.
OPTION_COMBINE_SHEETS_INTO_ONE_PDF = False # Default is False (separate PDF per sheet)

# Basic CSS for table styling
TABLE_CSS = """
@page {
    size: A4 landscape;
    margin: 0.75in;
}
body {
    font-family: 'Arial', sans-serif;
    font-size: 9pt;
}
table {
    width: 100%;
    border-collapse: collapse;
    margin-bottom: 15px;
}
th, td {
    border: 1px solid #cccccc;
    padding: 4px 6px;
    text-align: left;
    word-wrap: break-word;
}
th {
    background-color: #f2f2f2;
    font-weight: bold;
}
caption {
    caption-side: top;
    font-size: 1.2em;
    font-weight: bold;
    margin-bottom: 10px;
    text-align: left;
}
"""

def read_file_to_dataframes(file_path):
    """
    Reads an XLSX or CSV file and returns a dictionary of DataFrames.
    For XLSX, keys are sheet names. For CSV, key is 'data'.
    Returns None if the file type is unsupported or an error occurs.
    """
    dataframes = {}
    file_ext = os.path.splitext(file_path)[1].lower()

    try:
        if file_ext == ".xlsx":
            xls = pd.ExcelFile(file_path)
            if not xls.sheet_names:
                print(f"    INFO: No sheets found in XLSX file: {file_path}")
                return None
            for sheet_name in xls.sheet_names:
                dataframes[sheet_name] = xls.parse(sheet_name)
            print(f"    Successfully read {len(dataframes)} sheet(s) from {file_path}")
        elif file_ext == ".csv":
            df = pd.read_csv(file_path)
            dataframes['data'] = df # Use a generic key for CSV data
            print(f"    Successfully read CSV file: {file_path}")
        else:
            print(f"    WARNING: Unsupported file type: {file_path}")
            return None
        return dataframes
    except Exception as e:
        print(f"    ERROR: Could not read file {file_path}. Reason: {e}")
        return None

def convert_dataframes_to_pdf(dataframes_dict, original_file_path, base_pdf_path):
    """
    Converts a dictionary of dataframes to PDF(s).
    - dataframes_dict: {'sheet_name_or_data_key': pd.DataFrame}
    - original_file_path: Path of the source file (for context).
    - base_pdf_path: The base path for the output PDF (e.g., "output_dir/filename.pdf").
    """
    original_file_ext = os.path.splitext(original_file_path)[1].lower()

    try:
        if not dataframes_dict:
            print(f"    WARNING: No dataframes provided for {original_file_path}. Skipping PDF generation.")
            return

        # Determine if we should combine into one PDF or create separate PDFs
        # This logic is more relevant for XLSX. CSVs effectively have one "sheet".
        process_as_single_pdf = OPTION_COMBINE_SHEETS_INTO_ONE_PDF or len(dataframes_dict) == 1

        if process_as_single_pdf:
            all_html_content = ""
            for key, df in dataframes_dict.items():
                if df.empty:
                    print(f"      Sheet/Data '{key}' in {original_file_path} is empty. Skipping in combined PDF.")
                    continue
                
                # Use a more generic caption if it's a CSV or single sheet
                caption_text = f"Sheet: {key}" if original_file_ext == ".xlsx" and len(dataframes_dict) > 1 else os.path.basename(original_file_path)
                if original_file_ext == ".csv":
                    caption_text = f"Data from: {os.path.basename(original_file_path)}"

                table_html = f"<caption>{caption_text}</caption>" + df.to_html(index=False, escape=True)
                all_html_content += table_html
            
            if not all_html_content:
                print(f"    WARNING: All data in {original_file_path} was empty. No PDF generated.")
                return

            full_html = f"<html><head><meta charset='UTF-8'></head><body>{all_html_content}</body></html>"
            HTML(string=full_html).write_pdf(base_pdf_path, stylesheets=[CSS(string=TABLE_CSS)])
            print(f"    Successfully converted {original_file_path} (all data/sheets) to {base_pdf_path}")

        else: # Create separate PDF per sheet (primarily for multi-sheet XLSX if OPTION_COMBINE_SHEETS_INTO_ONE_PDF is False)
            base_name_for_multi, ext = os.path.splitext(base_pdf_path)
            for sheet_name, df in dataframes_dict.items():
                if df.empty:
                    print(f"      Sheet '{sheet_name}' in {original_file_path} is empty. Skipping this PDF.")
                    continue
                
                safe_sheet_name = "".join(c if c.isalnum() or c in (' ', '_', '-') else '_' for c in sheet_name).rstrip()
                sheet_pdf_path = f"{base_name_for_multi}_{safe_sheet_name}{ext}"
                
                html_content = df.to_html(index=False, escape=True)
                full_html = f"<html><head><meta charset='UTF-8'></head><body><caption>Sheet: {sheet_name}</caption>{html_content}</body></html>"
                HTML(string=full_html).write_pdf(sheet_pdf_path, stylesheets=[CSS(string=TABLE_CSS)])
                print(f"    Successfully converted {original_file_path} (sheet: {sheet_name}) to {sheet_pdf_path}")

    except Exception as e:
        print(f"    ERROR: Failed to generate PDF for {original_file_path}. Reason: {e}")
        import traceback
        traceback.print_exc()

def main():
    if not os.path.isdir(INPUT_DIR):
        print(f"ERROR: Input directory '{INPUT_DIR}' not found. Please create it and add files.")
        return

    os.makedirs(OUTPUT_PDF_DIR, exist_ok=True)
    print(f"Looking for files in: {os.path.abspath(INPUT_DIR)}")
    print(f"Attempting to convert file types: {', '.join(FILE_EXTENSIONS_TO_CONVERT)}")
    print(f"PDFs will be saved in: {os.path.abspath(OUTPUT_PDF_DIR)}\n")

    files_to_process = []
    print("--- Start File Discovery ---")
    if RECURSIVE_SEARCH:
        for root, dirs, files in os.walk(INPUT_DIR):
            print(f"  Scanning directory: {root}")
            # print(f"    Subdirectories found: {dirs}") # Optional: very verbose
            # print(f"    Files found: {files}")        # Optional: very verbose
            for file in files:
                # print(f"      Checking file: {file}") # Optional: very verbose
                file_ext = os.path.splitext(file)[1].lower()
                if file_ext in FILE_EXTENSIONS_TO_CONVERT:
                    if not file.startswith('~'): # Skip temporary Excel files
                        files_to_process.append(os.path.join(root, file))
                        print(f"    + Added for processing: {os.path.join(root, file)}")
                # else: # Optional: very verbose
                #     print(f"      - Skipped (wrong extension): {file}")

    else:
        print(f"  Scanning directory (non-recursive): {INPUT_DIR}")
        for ext in FILE_EXTENSIONS_TO_CONVERT:
            for file_path in glob.glob(os.path.join(INPUT_DIR, f"*{ext}")):
                 if not os.path.basename(file_path).startswith('~'): # Skip temporary Excel files
                    files_to_process.append(file_path)
                    print(f"    + Added for processing: {file_path}")
    print("--- End File Discovery ---\n")

    if not files_to_process:
        print("No files found to convert with the specified extensions.")
        return

    print(f"Found {len(files_to_process)} file(s) to process.\n")

    for file_path in files_to_process:
        print(f"Processing: {file_path}...")
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        pdf_filename = f"{base_name}.pdf"

        current_output_dir = OUTPUT_PDF_DIR
        if RECURSIVE_SEARCH and CREATE_SUBFOLDERS_IN_OUTPUT:
            relative_dir_path = os.path.relpath(os.path.dirname(file_path), INPUT_DIR)
            if relative_dir_path and relative_dir_path != '.':
                current_output_dir = os.path.join(OUTPUT_PDF_DIR, relative_dir_path)
                os.makedirs(current_output_dir, exist_ok=True)
        
        output_pdf_path = os.path.join(current_output_dir, pdf_filename)

        dataframes = read_file_to_dataframes(file_path)
        if dataframes:
            convert_dataframes_to_pdf(dataframes, file_path, output_pdf_path)
        else:
            print(f"  Skipping PDF generation for {file_path} due to read errors or no data.")
        print("-" * 40)

    print("\nBatch conversion finished.")

if __name__ == "__main__":
    main()

Looking for files in: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database
Attempting to convert file types: .xlsx, .csv
PDFs will be saved in: d:\OneDrive - Green Energy\Sakib\batch xlsx to pdf converter\output_pdf_files

--- Start File Discovery ---
  Scanning directory: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database
    + Added for processing: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\brand.csv
    + Added for processing: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\car_expense.csv
    + Added for processing: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\car_info.csv
    + Added for processing: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\connected_car.csv
    + Added for processing: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\designation.csv
    + Added for processing: D:\OneDrive - Green Energy\Sakib\personal-assistant-cha

In [ ]:
import os
import pandas as pd
from weasyprint import HTML, CSS
import glob

# --- Configuration ---
INPUT_DIR = r"D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database" # Your input
OUTPUT_PDF_DIR = "output_pdf_files"
RECURSIVE_SEARCH = True
CREATE_SUBFOLDERS_IN_OUTPUT = True
FILE_EXTENSIONS_TO_CONVERT = [".xlsx", ".csv"]

OPTION_COMBINE_SHEETS_INTO_ONE_PDF = False # For XLSX: True to combine, False for separate PDFs

# --- PDF STYLING CONFIGURATION (Tune these for wide tables!) ---
PAGE_SIZE = "A4 landscape"  # E.g., "A4 landscape", "A3 landscape", "letter landscape"
PAGE_MARGIN = "0.4in"       # Margins (e.g., "0.5in", "1cm")

BASE_FONT_SIZE_PT = 7       # START HERE: For many columns, try 6, 5, or even 4.
CELL_PADDING_PX = "2px 3px" # Padding in cells (e.g., "1px 2px" for very tight).
BODY_SCALE_FACTOR = 1.0     # Overall zoom: 0.9 = 90%, 0.8 = 80%. Use if font/padding isn't enough.
                            # Values < 0.7 can significantly degrade text quality.

# --- Generate Table CSS dynamically based on configuration ---
def get_table_css():
    body_transform_style = ""
    if isinstance(BODY_SCALE_FACTOR, (float, int)) and BODY_SCALE_FACTOR != 1.0 and 0.2 <= BODY_SCALE_FACTOR <= 2.0:
        body_transform_style = f"transform: scale({BODY_SCALE_FACTOR}); transform-origin: top left;"

    # Ensure caption font size is reasonably relative to base font size, but not too tiny
    caption_font_size_pt = max(5, BASE_FONT_SIZE_PT) # Caption at least 5pt, or same as base

    return f"""
    @page {{
        size: {PAGE_SIZE};
        margin: {PAGE_MARGIN};
    }}
    body {{
        font-family: 'Arial', 'Helvetica', sans-serif; /* Common fonts; use condensed fonts if available */
        font-size: {BASE_FONT_SIZE_PT}pt;
        {body_transform_style}
        -webkit-font-smoothing: antialiased;
        -moz-osx-font-smoothing: grayscale;
    }}
    table {{
        width: 100%; /* Table tries to use full page width (within margins) */
        border-collapse: collapse;
        margin-bottom: 10px;
        table-layout: auto; /* 'auto' is generally best for allowing dynamic column sizing */
                            /* 'fixed' could be an alternative if you were setting explicit column widths */
    }}
    th, td {{
        border: 1px solid #cccccc;
        padding: {CELL_PADDING_PX};
        text-align: left;
        word-wrap: break-word;     /* Standard word wrapping (breaks between words) */
        overflow-wrap: break-word; /* Alias for word-wrap */
        word-break: break-all;     /* CRITICAL FOR ADAPTIVE FIT: Breaks *within* words if needed */
        hyphens: auto;             /* Attempts to hyphenate text (language/font dependent) */
    }}
    th {{
        background-color: #f2f2f2;
        font-weight: bold;
    }}
    caption {{
        caption-side: top;
        font-size: {caption_font_size_pt}pt;
        font-weight: bold;
        margin-bottom: 8px;
        text-align: left;
    }}
    """

def read_file_to_dataframes(file_path):
    dataframes = {}
    file_ext = os.path.splitext(file_path)[1].lower()
    try:
        if file_ext == ".xlsx":
            xls = pd.ExcelFile(file_path)
            if not xls.sheet_names:
                print(f"    INFO: No sheets in XLSX: {file_path}")
                return None
            for sheet_name in xls.sheet_names:
                dataframes[sheet_name] = xls.parse(sheet_name)
            print(f"    Read {len(dataframes)} sheet(s) from {file_path}")
        elif file_ext == ".csv":
            encodings_to_try = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
            df = None
            for encoding in encodings_to_try:
                try:
                    df = pd.read_csv(file_path, encoding=encoding)
                    print(f"    Read CSV {file_path} with encoding {encoding}")
                    break
                except UnicodeDecodeError:
                    # print(f"    INFO: CSV {file_path} encoding {encoding} failed. Trying next...") # Verbose
                    pass
                except Exception as e_read:
                    print(f"    ERROR: Reading CSV {file_path} with encoding {encoding}. Reason: {e_read}")
                    break
            if df is None:
                 print(f"    ERROR: Failed to read CSV {file_path} with all attempted encodings.")
                 return None
            dataframes['data'] = df
        else:
            print(f"    WARNING: Unsupported file: {file_path}")
            return None
        return dataframes
    except Exception as e:
        print(f"    ERROR: Could not read file {file_path}. Reason: {e}")
        return None

def convert_dataframes_to_pdf(dataframes_dict, original_file_path, base_pdf_path, table_css_str):
    original_file_ext = os.path.splitext(original_file_path)[1].lower()
    try:
        if not dataframes_dict:
            print(f"    WARNING: No dataframes for {original_file_path}. Skipping PDF.")
            return

        process_as_single_pdf = OPTION_COMBINE_SHEETS_INTO_ONE_PDF or len(dataframes_dict) == 1 or original_file_ext == ".csv"

        if process_as_single_pdf:
            all_html_content = ""
            for key, df in dataframes_dict.items():
                if df.empty:
                    print(f"      Sheet/Data '{key}' in {original_file_path} is empty. Skipping.")
                    continue
                
                caption_text = os.path.basename(original_file_path)
                if original_file_ext == ".xlsx" and len(dataframes_dict) > 1:
                     caption_text = f"File: {os.path.basename(original_file_path)} - Sheet: {key}"
                elif original_file_ext == ".xlsx":
                     caption_text = f"File: {os.path.basename(original_file_path)} - Sheet: {key}"

                df_display = df.fillna('') # Replace NaN with empty string
                table_html = f"<caption>{caption_text}</caption>" + df_display.to_html(index=False, escape=True)
                all_html_content += table_html
            
            if not all_html_content:
                print(f"    WARNING: All data in {original_file_path} was empty. No PDF.")
                return

            full_html = f"<html><head><meta charset='UTF-8'></head><body>{all_html_content}</body></html>"
            HTML(string=full_html).write_pdf(base_pdf_path, stylesheets=[CSS(string=table_css_str)])
            print(f"    Successfully converted {original_file_path} to {base_pdf_path}")

        else: # Separate PDF per sheet
            base_name_for_multi, ext = os.path.splitext(base_pdf_path)
            for sheet_name, df in dataframes_dict.items():
                if df.empty:
                    print(f"      Sheet '{sheet_name}' in {original_file_path} is empty. Skipping.")
                    continue
                
                safe_sheet_name = "".join(c if c.isalnum() or c in (' ', '_', '-') else '_' for c in sheet_name).rstrip()
                sheet_pdf_path = f"{base_name_for_multi}_{safe_sheet_name}{ext}"
                
                df_display = df.fillna('')
                html_content = df_display.to_html(index=False, escape=True)
                full_html = f"<html><head><meta charset='UTF-8'></head><body><caption>Sheet: {sheet_name} from {os.path.basename(original_file_path)}</caption>{html_content}</body></html>"
                HTML(string=full_html).write_pdf(sheet_pdf_path, stylesheets=[CSS(string=table_css_str)])
                print(f"    Converted sheet '{sheet_name}' from {original_file_path} to {sheet_pdf_path}")

    except Exception as e:
        print(f"    ERROR: Failed to generate PDF for {original_file_path}. Reason: {e}")
        import traceback
        traceback.print_exc()

def main():
    if not os.path.isdir(INPUT_DIR):
        print(f"ERROR: Input directory '{INPUT_DIR}' not found.")
        return

    TABLE_CSS_STR = get_table_css() 

    os.makedirs(OUTPUT_PDF_DIR, exist_ok=True)
    print(f"Input directory: {os.path.abspath(INPUT_DIR)}")
    print(f"Output PDF directory: {os.path.abspath(OUTPUT_PDF_DIR)}")
    print(f"Converting types: {', '.join(FILE_EXTENSIONS_TO_CONVERT)}")
    print(f"Styling: Page '{PAGE_SIZE}' margin '{PAGE_MARGIN}', Font {BASE_FONT_SIZE_PT}pt, Padding '{CELL_PADDING_PX}', Scale {BODY_SCALE_FACTOR}")

    files_to_process = []
    # print("\n--- File Discovery ---") # Less verbose default
    if RECURSIVE_SEARCH:
        for root, _, files in os.walk(INPUT_DIR):
            for file in files:
                file_ext = os.path.splitext(file)[1].lower()
                if file_ext in FILE_EXTENSIONS_TO_CONVERT and not file.startswith('~'):
                    files_to_process.append(os.path.join(root, file))
    else:
        for ext in FILE_EXTENSIONS_TO_CONVERT:
            for file_path in glob.glob(os.path.join(INPUT_DIR, f"*{ext}")):
                 if not os.path.basename(file_path).startswith('~'):
                    files_to_process.append(file_path)
    
    if not files_to_process:
        print("No matching files found to convert.")
        return
    print(f"Found {len(files_to_process)} file(s). \n--- Processing ---")

    for file_path in files_to_process:
        print(f"Processing: {file_path}...")
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        pdf_filename = f"{base_name}.pdf"

        current_output_dir = OUTPUT_PDF_DIR
        if RECURSIVE_SEARCH and CREATE_SUBFOLDERS_IN_OUTPUT:
            relative_dir_path = os.path.relpath(os.path.dirname(file_path), INPUT_DIR)
            if relative_dir_path and relative_dir_path != '.':
                current_output_dir = os.path.join(OUTPUT_PDF_DIR, relative_dir_path)
                os.makedirs(current_output_dir, exist_ok=True)
        
        output_pdf_path = os.path.join(current_output_dir, pdf_filename)

        dataframes = read_file_to_dataframes(file_path)
        if dataframes:
            convert_dataframes_to_pdf(dataframes, file_path, output_pdf_path, TABLE_CSS_STR)
        else:
            print(f"  Skipping PDF for {file_path} (read error/no data).")
        print("-" * 40)

    print("\nBatch conversion finished.")

if __name__ == "__main__":
    main()

Input directory: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database
Output PDF directory: d:\OneDrive - Green Energy\Sakib\batch xlsx to pdf converter\output_pdf_files
Converting types: .xlsx, .csv
Styling: Page 'A4 landscape' margin '0.4in', Font 7pt, Padding '2px 3px', Scale 1.0
Found 24 file(s). 
--- Processing ---
Processing: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\brand.csv...
    Read CSV D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\brand.csv with encoding utf-8
    Successfully converted D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\brand.csv to output_pdf_files\brand.pdf
----------------------------------------
Processing: D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\car_expense.csv...
    Read CSV D:\OneDrive - Green Energy\Sakib\personal-assistant-chatbot\database\car_expense.csv with encoding utf-8
      Sheet/Data 'data' in D:\OneDrive - Green Energy\Sakib\p